In [2]:
import pandas as pd 
import numpy as np

In [14]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import precision_score
import tensorflow as tf

In [ ]:
# Loading in the Data & splitting it into variables 
data = np.load('disprot_esm_embed_10188.npz')
# feature data
x_data = data['X']
# ground truth 
y_data = data['y']

In [19]:
x_data.shape

(10188, 1280)

In [24]:
# splitting the data into training and testing data 
x_train, x_test, y_train, y_test = train_test_split(
                                   x_data, y_data, test_size=0.20, 
                                   random_state=42)

In [222]:
# Creating the Neural Network
myModel = tf.keras.Sequential() 
myModel.add(tf.keras.layers.Dense(units=15, input_shape=(1280,), activation='relu'))
myModel.add(tf.keras.layers.Dense(units=10, activation='sigmoid'))
myModel.add(tf.keras.layers.Dense(units=5, activation='sigmoid'))
myModel.add(tf.keras.layers.Dense(units=1, activation='relu'))

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [223]:
myModel.compile(loss='binary_crossentropy')

In [224]:
myModel.fit(x_train,y_train)

255/255 ━━━━━━━━━━━━━━━━━━━━ 0s 520us/step - loss: 0.6067


In [225]:
predictionRaw = myModel.predict(x_test) 
predictionRaw

64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 637us/step


array([[0.05170798],
       [0.8781096 ],
       [0.9265172 ],
       ...,
       [0.        ],
       [0.36352998],
       [0.88083947]], dtype=float32)

In [226]:
predictionProbability = predictionRaw.flatten()
prediction = predictionProbability.round()


In [227]:
test_y = pd.DataFrame(y_test)

In [228]:
performanceDataFrame = pd.DataFrame(
                       data = {
                               'Prediction':prediction,
                               'Prediction_Probability':predictionProbability,
                               'Ground_Truth':test_y[0]
                               })

performanceDataFrame['90_Probability'] = (performanceDataFrame['Prediction_Probability'] >= 0.90).astype(int)
performanceDataFrame['10_Probability'] = (performanceDataFrame['Prediction_Probability'] >= 0.10).astype(int)

In [242]:
display(performanceDataFrame)
performanceDataFrame[performanceDataFrame['Prediction_Probability'] >= .90]

,Prediction,Prediction_Probability,Ground_Truth,90_Probability,10_Probability
0,0.0,0.051708,0.0,0,0
1,1.0,0.878110,1.0,0,1
2,1.0,0.926517,1.0,1,1
3,0.0,0.176347,0.0,0,1
4,1.0,0.554340,1.0,0,1
...,...,...,...,...,...
2033,0.0,0.394478,1.0,0,1
2034,1.0,0.940385,1.0,1,1
2035,0.0,0.000000,0.0,0,0
2036,0.0,0.363530,0.0,0,1


,Prediction,Prediction_Probability,Ground_Truth,90_Probability,10_Probability
2,1.0,0.926517,1.0,1,1
78,1.0,0.981818,1.0,1,1
94,1.0,0.997036,1.0,1,1
141,1.0,1.014161,1.0,1,1
154,1.0,0.922593,1.0,1,1
...,...,...,...,...,...
1981,1.0,0.979254,1.0,1,1
1998,1.0,0.934105,1.0,1,1
2006,1.0,0.918175,1.0,1,1
2015,1.0,0.957708,1.0,1,1


In [236]:
crosstabpred = pd.crosstab(performanceDataFrame['Prediction'],
                           performanceDataFrame['Ground_Truth'], margins= False)
crosstab10 = pd.crosstab(performanceDataFrame['10_Probability'],
                         performanceDataFrame['Ground_Truth'], margins= False)

crosstab90 = pd.crosstab(performanceDataFrame['90_Probability'],
                         performanceDataFrame['Ground_Truth'], margins= False)

In [237]:
display(crosstabpred, crosstab10, crosstab90)

Ground_Truth,0.0,1.0
Prediction,,
0.0,1399,191
1.0,57,391


Ground_Truth,0.0,1.0
10_Probability,,
0,641,27
1,815,555


Ground_Truth,0.0,1.0
90_Probability,,
0,1456,482
1,0,100


### **General prediction** has a relatively low false positive rate and has a false negative rate that could be improved if made less conservative

### **10% probability** has low false negatives but very high false positive rate 
### **90% probability** has no false positives but a sizeable amount of false negatives